# 서울시 미용실 데이터 - NAVER API 데이터 수집
## 작성자(최종수정일) : 서동원(2020.04.11)
- 서울 열린데이터 광장 API를 이용해 수집한 데이터(이하 공공데이터) 활용
- 공공데이터를 네이버 지역검색 API에 적용시켜 네이버에 등록된 미용실 데이터 수집

In [741]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import re
import requests
import time
import datetime
pd.set_option('display.max_columns',50) # DataFrame truncation 없이 보기

from tqdm import tnrange
from tqdm import tqdm

## (1) 공공데이터 가져와 전처리 하기

- 이하 전처리 작업에서 매번 공공 API를 호출할 필요 없다.
- 공공 API로 호출한 데이터를 저장한 csv파일을 읽어와 진핼할 것
- 일단 2020년 4월 9일자 기준 데이터로 전처리를 진행하겠다.

In [742]:
# csv 파일 읽어서 DF에 대입
public_salon_raw = pd.read_csv('./public_api_seoul_salon_data_2020-04-09.csv', index_col=0)

In [743]:
# 데이터 확인
public_salon_raw.head(2)

,지역,CGG_CODE,SNT_COB_CODE,YY,UPSO_SNO,SNT_COB_NM,UPSO_GSL_YMD,UPSO_NM,TRDP_AREA,UPSO_SITE_TELNO,BMAN_STDT,BUP_NM,SITE_STDT,ADMDNG_NM,DCB_YMD,DCB_WHY,SNT_UPTAE_NM,ED_FIN_YMD,GAEKSIL,HANSHIL,YANGSIL,CHAIR_NUM,YOKSIL,BALHANSIL_YN,WASHMC_NUM,PERM_NT_NO,KOR_FRGNR_GBN,NTN,SITE_ADDR_RD,SITE_ADDR
0,강남구,3220000,211,1983,1,미용업(일반),19830627,백민재헤어샵,19.80,02 5451290,20110502,NaN,20180515,청담동,NaN,NaN,일반미용업,20110622.0,0.0,0.0,0.0,3.0,0.0,N,0.0,3220000-211-1983-00001,내국인,NaN,"서울특별시 강남구 영동대로 702, 화천회관빌딩 지상2층 222-1호 (청담동)",서울특별시 강남구 청담동 133번지 3호 화천회관빌딩
1,강남구,3220000,211,1983,1,미용업(일반),19830627,백민재헤어샵,65.67,02 5451290,20110502,NaN,20180515,청담동,NaN,NaN,일반미용업,20110622.0,0.0,0.0,0.0,3.0,0.0,N,0.0,3220000-211-1983-00001,내국인,NaN,"서울특별시 강남구 영동대로 702, 화천회관빌딩 지상2층 222-1호 (청담동)",서울특별시 강남구 청담동 133번지 3호 화천회관빌딩


In [744]:
# 컬럼을 보기 쉽게 변환하기
# 컬럼명 정보가 담긴 csv 파일 읽어오기 
col_convert = pd.read_csv('./public_api_column_info.csv')
col_convert

,출력명,출력설명
0,CGG_CODE,시군구코드
1,SNT_COB_CODE,업종코드
2,YY,년도
3,UPSO_SNO,업소일련번호
4,SNT_COB_NM,업종명
5,UPSO_GSL_YMD,신고일자
6,UPSO_NM,업소명
7,TRDP_AREA,면적
8,UPSO_SITE_TELNO,소재지전화번호
9,BMAN_STDT,영업자시작일


In [745]:
# 두 컬럼의 값을 dict 객체로 변환
col_dict = dict(zip(col_convert.출력명, col_convert.출력설명))
col_dict

{'CGG_CODE': '시군구코드',
 'SNT_COB_CODE': '업종코드',
 'YY': '년도',
 'UPSO_SNO': '업소일련번호',
 'SNT_COB_NM': '업종명',
 'UPSO_GSL_YMD': '신고일자',
 'UPSO_NM': '업소명',
 'TRDP_AREA': '면적',
 'UPSO_SITE_TELNO': '소재지전화번호',
 'BMAN_STDT': '영업자시작일',
 'BUP_NM': '법인명',
 'SITE_STDT': '소재지시작일',
 'ADMDNG_NM': '행정동명',
 'DCB_YMD': '폐업일자',
 'DCB_WHY': '폐업사유',
 'SNT_UPTAE_NM': '업태명',
 'ED_FIN_YMD': '위생교육수료일자',
 'GAEKSIL': '객실수',
 'HANSHIL': '한실수',
 'YANGSIL': '양실수',
 'CHAIR_NUM': '의자수',
 'YOKSIL': '욕실수',
 'BALHANSIL_YN': '발한실',
 'WASHMC_NUM': '세탁기수',
 'PERM_NT_NO': '허가(신고)번호',
 'KOR_FRGNR_GBN': '내외국인구분',
 'NTN': '국적',
 'SITE_ADDR_RD': '소재지도로명',
 'SITE_ADDR': '소재지지번'}

In [746]:
# 컬럼명 변경하기
public_salon_raw.rename(columns=col_dict, inplace=True)
public_salon_raw.head(1)

,지역,시군구코드,업종코드,년도,업소일련번호,업종명,신고일자,업소명,면적,소재지전화번호,영업자시작일,법인명,소재지시작일,행정동명,폐업일자,폐업사유,업태명,위생교육수료일자,객실수,한실수,양실수,의자수,욕실수,발한실,세탁기수,허가(신고)번호,내외국인구분,국적,소재지도로명,소재지지번
0,강남구,3220000,211,1983,1,미용업(일반),19830627,백민재헤어샵,19.8,02 5451290,20110502,NaN,20180515,청담동,NaN,NaN,일반미용업,20110622.0,0.0,0.0,0.0,3.0,0.0,N,0.0,3220000-211-1983-00001,내국인,NaN,"서울특별시 강남구 영동대로 702, 화천회관빌딩 지상2층 222-1호 (청담동)",서울특별시 강남구 청담동 133번지 3호 화천회관빌딩


In [747]:
public_salon_raw.tail(1)

,지역,시군구코드,업종코드,년도,업소일련번호,업종명,신고일자,업소명,면적,소재지전화번호,영업자시작일,법인명,소재지시작일,행정동명,폐업일자,폐업사유,업태명,위생교육수료일자,객실수,한실수,양실수,의자수,욕실수,발한실,세탁기수,허가(신고)번호,내외국인구분,국적,소재지도로명,소재지지번
28286,중랑구,3060000,211,2020,10,미용업(일반),20200310,우주헤어,19.65,NaN,20200310,NaN,20200310,상봉제1동,NaN,NaN,일반미용업,NaN,0.0,0.0,0.0,4.0,0.0,N,0.0,3060000-211-2020-00010,내국인,NaN,"서울특별시 중랑구 망우로43길 3, 1층 좌측호 (상봉동)",서울특별시 중랑구 상봉동 116번지 45호


In [748]:
# 총 데이터 개수 확인
pre_data_count = len(public_salon_raw)
print('총 데이터 수 :', len(public_salon_raw))

총 데이터 수 : 28287


---
- 업소명에 '?'문자가 있는 부분을 조사해보니 '샾'을 인식하지 못해 물음표로 처리된 것으로 판단.
- '?' 문자를 '샵'으로 변환

In [749]:
# 업소명에 '?' 문자를 '샵' 문자로 치환하기
public_salon_raw['업소명'] = public_salon_raw['업소명'].apply(lambda x: re.sub('[？]', '샵', x))


# '?'문자가 '샵'으로 전부 변환되지 않는 문제를 발견. 조치 필요 !!!!
# 일반 문자'?' 가 아닌 특수문자'？' 이어서 대체가 안된 것

---
- 중복값 처리

In [750]:
# 모든 컬럼값이 중복인 row 제거
public_salon_raw.drop_duplicates(inplace=True)

# 중복 제거 후 데이터 개수 확인
print('중복여부로 제거 전 데이터 수 :', pre_data_count)
print('중복여부로 제거 후 데이터 수 :', len(public_salon_raw))
print('중복여부로 제거된 데이터 수 :', pre_data_count - len(public_salon_raw))
pre_data_count = len(public_salon_raw)

중복여부로 제거 전 데이터 수 : 28287
중복여부로 제거 후 데이터 수 : 28231
중복여부로 제거된 데이터 수 : 56


---
- 폐업점 데이터 처리

In [751]:
# '폐업일자'의 값이 NaN이면 영업중
# '폐업일자'의 값이 NaN이 아니면 폐업된 것으로 판단

# '폐업일자'에 값이 존재하는 폐업된 업장이 얼마나 있는지 확인 
public_salon_raw[public_salon_raw['폐업일자'].notnull()]['폐업일자']

5        20200113.0
7        20130829.0
8        20150821.0
21       20180222.0
25       20190415.0
            ...    
28206    20190827.0
28227    20191119.0
28258    20190802.0
28265    20200213.0
28268    20200206.0
Name: 폐업일자, Length: 8616, dtype: float64

In [752]:
# 폐업한 미용실 정보가 8591개나 된다.

In [753]:
# '폐업일자' 컬럼이 NaN인, 영업중인 행만 불리언 색인 -> public_salon_raw에 대입
public_salon_raw = public_salon_raw[public_salon_raw['폐업일자'].isnull()]

# 데이터 개수 확인

# 폐업여부로 삭제 후 데이터 개수 확인
print('폐업여부로 제거 전 데이터 수 :', pre_data_count)
print('폐업여부로 제거된 데이터 수 :', pre_data_count - len(public_salon_raw))
print('폐업여부로 제거 후 데이터 수 :', len(public_salon_raw))
pre_data_count = len(public_salon_raw)

폐업여부로 제거 전 데이터 수 : 28231
폐업여부로 제거된 데이터 수 : 8616
폐업여부로 제거 후 데이터 수 : 19615


In [754]:
# 업태명에 어떤 값들이 있는지 확인
public_salon_raw['업태명'].unique()

array(['일반미용업', '네일아트업', '기타', '메이크업업', '일반이용업', '숙박업 기타'], dtype=object)

In [755]:
# '업태명'이 '일반미용업', '일반이용업' 인 데이터만 남기기
public_salon_raw = public_salon_raw[public_salon_raw['업태명'].isin(['일반미용업', '일반이용업'])]

# 업태명으로 삭제 후 데이터 개수 확인
print('업태명으로 제거 전 데이터 수 :', pre_data_count)
print('업태명으로 제거 후 데이터 수 :', len(public_salon_raw))
print('업태명으로 제거된 데이터 수 :', pre_data_count - len(public_salon_raw))
pre_data_count = len(public_salon_raw)

업태명으로 제거 전 데이터 수 : 19615
업태명으로 제거 후 데이터 수 : 19355
업태명으로 제거된 데이터 수 : 260


In [756]:
# 컬럼명 확인하기
public_salon_raw.columns

Index(['지역', '시군구코드', '업종코드', '년도', '업소일련번호', '업종명', '신고일자', '업소명', '면적',
       '소재지전화번호', '영업자시작일', '법인명', '소재지시작일', '행정동명', '폐업일자', '폐업사유', '업태명',
       '위생교육수료일자', '객실수', '한실수', '양실수', '의자수', '욕실수', '발한실', '세탁기수',
       '허가(신고)번호', '내외국인구분', '국적', '소재지도로명', '소재지지번'],
      dtype='object')

In [757]:
public_salon_raw[public_salon_raw['업태명'] == '일반이용업']

,지역,시군구코드,업종코드,년도,업소일련번호,업종명,신고일자,업소명,면적,소재지전화번호,영업자시작일,법인명,소재지시작일,행정동명,폐업일자,폐업사유,업태명,위생교육수료일자,객실수,한실수,양실수,의자수,욕실수,발한실,세탁기수,허가(신고)번호,내외국인구분,국적,소재지도로명,소재지지번
5474,강서구,3150000,211,2017,26,미용업(일반),20170515,승민헤어,97.85,0262057133,20170515,NaN,20180903,화곡제2동,NaN,NaN,일반이용업,NaN,0.0,0.0,0.0,4.0,0.0,N,0.0,3150000-211-2017-00026,내국인,NaN,"서울특별시 강서구 강서로8길 68, 1층 (화곡동)",서울특별시 강서구 화곡동 882번지 26호 (지상 1층)
5475,강서구,3150000,211,2017,26,미용업(일반),20170515,승민헤어,24.00,0262057133,20170515,NaN,20180903,화곡제2동,NaN,NaN,일반이용업,NaN,0.0,0.0,0.0,4.0,0.0,N,0.0,3150000-211-2017-00026,내국인,NaN,"서울특별시 강서구 강서로8길 68, 1층 (화곡동)",서울특별시 강서구 화곡동 882번지 26호 (지상 1층)
5476,강서구,3150000,211,2017,26,미용업(일반),20170515,규림헤어겔러리,97.85,0262057133,20170515,NaN,20180903,화곡제2동,NaN,NaN,일반이용업,NaN,0.0,0.0,0.0,4.0,0.0,N,0.0,3150000-211-2017-00026,내국인,NaN,"서울특별시 강서구 강서로8길 68, 1층 (화곡동)",서울특별시 강서구 화곡동 882번지 26호 (지상 1층)
5477,강서구,3150000,211,2017,26,미용업(일반),20170515,규림헤어겔러리,24.00,0262057133,20170515,NaN,20180903,화곡제2동,NaN,NaN,일반이용업,NaN,0.0,0.0,0.0,4.0,0.0,N,0.0,3150000-211-2017-00026,내국인,NaN,"서울특별시 강서구 강서로8길 68, 1층 (화곡동)",서울특별시 강서구 화곡동 882번지 26호 (지상 1층)
5478,강서구,3150000,211,2017,26,미용업(일반),20170515,승민헤어,97.85,0262057133,20170515,NaN,20170515,화곡제8동,NaN,NaN,일반이용업,NaN,0.0,0.0,0.0,4.0,0.0,N,0.0,3150000-211-2017-00026,내국인,NaN,"서울특별시 강서구 곰달래로25길 77, 2층 (화곡동)",서울특별시 강서구 화곡동 340번지 59호 2층
5479,강서구,3150000,211,2017,26,미용업(일반),20170515,승민헤어,24.00,0262057133,20170515,NaN,20170515,화곡제8동,NaN,NaN,일반이용업,NaN,0.0,0.0,0.0,4.0,0.0,N,0.0,3150000-211-2017-00026,내국인,NaN,"서울특별시 강서구 곰달래로25길 77, 2층 (화곡동)",서울특별시 강서구 화곡동 340번지 59호 2층
5480,강서구,3150000,211,2017,26,미용업(일반),20170515,규림헤어겔러리,97.85,0262057133,20170515,NaN,20170515,화곡제8동,NaN,NaN,일반이용업,NaN,0.0,0.0,0.0,4.0,0.0,N,0.0,3150000-211-2017-00026,내국인,NaN,"서울특별시 강서구 곰달래로25길 77, 2층 (화곡동)",서울특별시 강서구 화곡동 340번지 59호 2층
5481,강서구,3150000,211,2017,26,미용업(일반),20170515,규림헤어겔러리,24.00,0262057133,20170515,NaN,20170515,화곡제8동,NaN,NaN,일반이용업,NaN,0.0,0.0,0.0,4.0,0.0,N,0.0,3150000-211-2017-00026,내국인,NaN,"서울특별시 강서구 곰달래로25길 77, 2층 (화곡동)",서울특별시 강서구 화곡동 340번지 59호 2층
6204,관악구,3200000,211,2008,1,미용업(일반),20080708,신보배 헤어,28.05,02 883 9489,20171026,NaN,20080708,은천동,NaN,NaN,일반이용업,NaN,0.0,0.0,0.0,5.0,0.0,NaN,0.0,3200000-211-2008-00001,내국인,NaN,"서울특별시 관악구 은천로 95, 벽산블루밍상가 501동 308호 (봉천동)",서울특별시 관악구 봉천동 1718번지 벽산블루밍상가
6205,관악구,3200000,211,2008,1,미용업(일반),20080708,머리못하는집,28.05,02 883 9489,20171026,NaN,20080708,은천동,NaN,NaN,일반이용업,NaN,0.0,0.0,0.0,5.0,0.0,NaN,0.0,3200000-211-2008-00001,내국인,NaN,"서울특별시 관악구 은천로 95, 벽산블루밍상가 501동 308호 (봉천동)",서울특별시 관악구 봉천동 1718번지 벽산블루밍상가


In [758]:
# 필요한 컬럼들로만 재색인
public_salon = public_salon_raw.reindex(columns=['지역'
                                                 ,'업소명'
                                                 ,'소재지전화번호'
                                                 ,'행정동명'
                                                 ,'소재지도로명'
                                                 ,'소재지지번'
                                                ])
public_salon.head(3)

,지역,업소명,소재지전화번호,행정동명,소재지도로명,소재지지번
0,강남구,백민재헤어샵,02 5451290,청담동,"서울특별시 강남구 영동대로 702, 화천회관빌딩 지상2층 222-1호 (청담동)",서울특별시 강남구 청담동 133번지 3호 화천회관빌딩
1,강남구,백민재헤어샵,02 5451290,청담동,"서울특별시 강남구 영동대로 702, 화천회관빌딩 지상2층 222-1호 (청담동)",서울특별시 강남구 청담동 133번지 3호 화천회관빌딩
2,강남구,백민재헤어샵,02 5451290,삼성1동,"서울특별시 강남구 삼성로 649, (삼성동,(상아APT 상가 107호))",서울특별시 강남구 삼성동 19번지 4호 (상아APT 상가 107호)


In [759]:
# DF의 head()로 보기에는 문제가 없지만
# 직접 찍어보니 공백이 여러개 중복되어 있는 경우가 있다.
public_salon['소재지지번'][0]

'서울특별시 강남구 청담동  133번지 3호  화천회관빌딩'

## 공백 제거
- 각 컬럼에 들어있는 값(문자열)의 
    - (1) 중복 공백을 1개로 줄인 뒤
    - (2) 앞뒤 공백 제거

In [760]:
# '업소명' 
public_salon['업소명'] = public_salon['업소명'].apply(lambda x: re.sub('\s+', ' ', x).strip())

# '소재지전화번호' 
public_salon['소재지전화번호'] = public_salon['소재지전화번호'].apply(lambda x: re.sub('\s+', ' ', str(x)).strip() if x else x)
# public_salon['소재지전화번호'] = public_salon['소재지전화번호'].apply(lambda x: remove_double_whitespace(x) if x else x)

# '행정동명' 
public_salon['행정동명'] = public_salon['행정동명'].apply(lambda x: re.sub('\s+', ' ', str(x)).strip() if x else x)
# public_salon['행정동명'] = public_salon['행정동명'].apply(lambda x: remove_double_whitespace(x) if x else x)

# '소재지도로명' 
public_salon['소재지도로명'] = public_salon['소재지도로명'].apply(lambda x: re.sub('\s+', ' ', str(x)).strip() if x else x)
# public_salon['소재지도로명'] = public_salon['소재지도로명'].apply(lambda x: remove_double_whitespace(x) if x else x)

# '소재지지번' 
public_salon['소재지지번'] = public_salon['소재지지번'].apply(lambda x: re.sub('\s+', ' ', str(x)).strip() if x else x)
# public_salon['소재지지번'] = public_salon['소재지지번'].apply(lambda x: remove_double_whitespace(x) if x else x)

In [761]:
# 작업 확인
public_salon['소재지지번'][0]

'서울특별시 강남구 청담동 133번지 3호 화천회관빌딩'

In [762]:
pre_data = len(public_salon)

## 중복 제거
- raw 데이터에서 필요한 컬럼만 재색인 했는데 다시 한번 중복 제거를 하겠다.

In [763]:
# 중복 제거
public_salon.drop_duplicates(inplace=True)

print('중복 제거 전 데이터 수 :', pre_data)
print('중복 제거된 데이터 수 :', pre_data - len(public_salon))
print('중복 제거 후 데이터 수 :', len(public_salon))

중복 제거 전 데이터 수 : 19355
중복 제거된 데이터 수 : 3441
중복 제거 후 데이터 수 : 15914


In [764]:
# index 빠진 이빨 채우기
# index를 0부터 데이터 길이만큼 재설정
public_salon.index=range(len(public_salon))

In [765]:
# 데이터 개수 확인
len(public_salon)

15914

In [766]:
# 데이터(인덱스가 잘 변경 됐는지) 확인
# 데이터길이와 마지막 인덱스 비교 
public_salon.tail(3)

,지역,업소명,소재지전화번호,행정동명,소재지도로명,소재지지번
15911,중랑구,에이데이즈 헤어살롱,nan,신내1동,"서울특별시 중랑구 봉화산로 232, 2층 (신내동)",서울특별시 중랑구 신내동 405번지 5호
15912,중랑구,지노헤어 상봉점,nan,상봉제2동,"서울특별시 중랑구 망우로 282-1, (상봉동)",서울특별시 중랑구 상봉동 115번지 14호
15913,중랑구,우주헤어,nan,상봉제1동,"서울특별시 중랑구 망우로43길 3, 1층 좌측호 (상봉동)",서울특별시 중랑구 상봉동 116번지 45호


In [767]:
# '소재지도로명'에 누락값 있는지 확인
public_salon[public_salon['소재지도로명'].isnull()]

,지역,업소명,소재지전화번호,행정동명,소재지도로명,소재지지번


In [768]:
# '소재지지번'에 누락값 있는지 확인
print("'소재지지번'이 누락된 데이터 수 :", len(public_salon[public_salon['소재지지번'].isnull()]))
public_salon[public_salon['소재지지번'].isnull()]

'소재지지번'이 누락된 데이터 수 : 0


,지역,업소명,소재지전화번호,행정동명,소재지도로명,소재지지번


- 특이사항
- 소재지도로명'과 '소재지지번'의 누락값이 일부 발견되었으나
- 두 컬럼의 값이 모두 누락된 경우는 없음
- 특이사항으로 '소재지지번'이 누락된 경우 '행정동명' 컬럼 또한 같이 누락됨

In [769]:
# 네이버 API 검색에 사용할 '검색어' 컬럼 추가
# '서울시 지역구 행정동 업소명 미용실' 형태로 저장
public_salon['검색어'] = list(map(lambda x: '서울시 %s %s 미용실' % (x[0], x[1]), public_salon.values))

In [770]:
# 데이터 확인
public_salon.tail()

,지역,업소명,소재지전화번호,행정동명,소재지도로명,소재지지번,검색어
15909,중랑구,살롱 드 로이,nan,묵제1동,"서울특별시 중랑구 동일로 942, 1층 102호 (묵동, 위더스빌)",서울특별시 중랑구 묵동 169번지 11호 위더스빌,서울시 중랑구 살롱 드 로이 미용실
15910,중랑구,기나 헤어샵,nan,중화제1동,"서울특별시 중랑구 동일로130길 52, 건형빌딩 1층 102호 (중화동)",서울특별시 중랑구 중화동 148번지 126호 건형빌딩,서울시 중랑구 기나 헤어샵 미용실
15911,중랑구,에이데이즈 헤어살롱,nan,신내1동,"서울특별시 중랑구 봉화산로 232, 2층 (신내동)",서울특별시 중랑구 신내동 405번지 5호,서울시 중랑구 에이데이즈 헤어살롱 미용실
15912,중랑구,지노헤어 상봉점,nan,상봉제2동,"서울특별시 중랑구 망우로 282-1, (상봉동)",서울특별시 중랑구 상봉동 115번지 14호,서울시 중랑구 지노헤어 상봉점 미용실
15913,중랑구,우주헤어,nan,상봉제1동,"서울특별시 중랑구 망우로43길 3, 1층 좌측호 (상봉동)",서울특별시 중랑구 상봉동 116번지 45호,서울시 중랑구 우주헤어 미용실


---
# (2) 네이버 지도 API 이용 - 데이터 크롤링

In [771]:
import requests
import datetime
import time
import math

In [772]:
# 네이버 API id와 secret이 저장된 csv파일 읽어오기
naver_api_key_info = pd.read_csv('./naver_api_key_info.csv')
naver_api_key_info

,client_id,client_secret,master
0,sDYPCi0TlT6FFoKVbOTz,lyX1n9_ht3,dongwon
1,c4xoBIW3TDfMkx0tIWhY,5559Q2hcvL,taekho
2,2c2rrojUSlLuh1tg7iaF,ugkxWiUe37,juri
3,q7SykvattsUjA2I5V2Cg,hPUw6Jm47t,minho
4,NN9VL8iSvJr1q6G_xd8k,neFbxVgRKr,daniel_lee
5,Q_XeKchRtiH_GoiVFpHb,fZmSGIYd2D,minho2


### 사용할 id와 secret 지정해주기

In [773]:
# master 지정
master = 'dongwon'
id_secret_array = naver_api_key_info[naver_api_key_info['master'] == master][['client_id', 'client_secret']].values[0]

# 네이버 API 인증 ID
client_id = id_secret_array[0]
# 네이버 API 인증 PW
client_secret = id_secret_array[1]

print('client_id :', client_id)
print('client_secret :', client_secret)

client_id : sDYPCi0TlT6FFoKVbOTz
client_secret : lyX1n9_ht3


### 네이버 API로 지도검색 정보를 요청하는 함수

In [774]:
# request_naver_api(검색어, 시작번호, 조회개수)

def request_naver_api(keyword, start, display):
    
    # 기본 URL
    url = 'https://openapi.naver.com/v1/search/local.json'
    
    # 헤더에 담을 인증키 정보
    header = {'X-Naver-Client-Id' : client_id, 'X-Naver-Client-Secret' : client_secret }

    # 쿼리스트링
    queryString = {
          'query' : keyword
        , 'start' : str(start)
        , 'display' : str(display)
    }

    # request 보내서 response_naver에 저장
    response_naver = requests.get(url, headers=header, params = queryString)
    
    try:
        if response_naver.status_code == 200: # 정상처리되면
            print ("[%s] URL Request SUCCESS" % datetime.datetime.now()) # 정상출력
            result_dict = response_naver.json()  # 실행결과 dict 객체로 변환
            return result_dict  # dict 객체 반환
        else: # 에러 발생시
            print("[%s] URL Request FAIL(type 1)" % datetime.datetime.now()) # 요청 실패 출력
            print('-' * 10, 'Error Message', '-' * 10 )
            print('Status Code :', response_naver.status_code)
            result_dict = response_naver.json()
            if result_dict:
                print('Error Code :', result_dict['errorCode'], result_dict['errorMessage'])
                print('-' * 35)
            return None # 에러 발생시 None 리턴
    except Exception as e:
        print("[%s] URL Request FAIL(type 2)" % datetime.datetime.now()) # 요청 실패 출력
        print('-' * 10, 'Error Message', '-' * 10 )
        print(e.with_traceback)
        print(e)
        print('-' * 35)
        return None # 에러 발생시 None 리턴

In [775]:
# 네이버 검색 결과를 별도 저장할 DF 생성
# (첫번째 실행때만 DF 생성 코드 실행할 것)
naver_search_all = DataFrame(columns=['업소명'
                                    , '소재지전화번호'
                                    , '소재지도로명'
                                    , '소재지지번'
                                    , '카테고리'
                                    , '검색인덱스'
                                    , '검색어'
                                    , '검색건수'
                                    ])

naver_search_all

# # 두번째 실행부터는 저장된 csv파일 불러오기
# # 저장된 네이버 검색 결과 csv 파일 불러와 DF 생성
# naver_search_all = pd.read_csv('./naver_api_search_data_YYYY-MM-DD.csv', encoding='utf-8').drop(columns='Unnamed: 0')
# naver_search_all

,업소명,소재지전화번호,소재지도로명,소재지지번,카테고리,검색인덱스,검색어,검색건수


In [776]:
# 첫번째 실행때만 DF 생성 코드 실행할 것
# 검색결과가 300건이 넘는 경우 api 사용 중 에러 발생.
# 이를 검색하지않고 따로 저장하는 DF 생성
naver_search_rejected = DataFrame(columns=['지역'
                                         , '업소명'
                                         , '소재지도로명'
                                         , '검색어'
                                         , '검색건수'
                                         , '원인'
                                         ])
naver_search_rejected

# # 두번째 실행부터는 저장된 csv 파일 불러오기
# # 검색결과 300건이 넘어 크롤링을 보류한 데이터 csv 파일 불러와 DF 생성
# naver_search_rejected = pd.read_csv('./naver_api_search_rejected_data_YYYY-MM-DD.csv', encoding='utf-8').drop(columns='Unnamed: 0')
# naver_search_rejected

,지역,업소명,소재지도로명,검색어,검색건수,원인


In [777]:
search_keyword_list = list(public_salon['검색어'])
search_keyword_list[:6]

['서울시 강남구 백민재헤어샵 미용실',
 '서울시 강남구 백민재헤어샵 미용실',
 '서울시 강남구 들어가도싱글나가도벙글 미용실 미용실',
 '서울시 강남구 조희미용실 미용실',
 '서울시 강남구 정정원헤어룩 미용실',
 '서울시 강남구 박승철헤어스투디오 청담점 미용실']


## 아래 함수는...
- "업소명"으로 검색 -> 검색된 모든 데이터 DF으로 변환
- 변환 후 csv 파일로 저장


In [778]:
skip_history = 0

In [779]:
def run_naver_api_search(start_number, end_number, restart_number):
    
    # 재시작 번호가 0이 아니면 시작번호를 재시작번호로 변경
    if restart_number != 0:
        start_number = restart_number
    
    global skip_history  # skip history 전역변수 사용
    global search_keyword_list  # 검색어 리스트 전역변수 사용
    global naver_search_all  # 검색결과 저장 DF 전역변수 사용
    global naver_search_rejected  # 검색거부 저장 DF 전역변수 사용
    
    count_test_error=0  # test connection 에러 카운트 변수
    
    for i in tqdm(range(start_number, end_number + 1)):

        search_keyword = search_keyword_list[i]

        # times.leep() 에 사용할 랜덤값 생성
        randtime = (np.random.rand(1)[0] + 0.5) * 2
        
        # interval 설정 (1~3 초 사이값)
        time.sleep(randtime)
        
        print('Try row-index :', i)
        print('검색어 :', search_keyword) # 검색어 출력

        # 총 검색결과 수를 알아내기 위해 '서울 [업소명]'으로 검색, 1번 데이터부터, 1개 요청
        print('<test connection>')
        dict_result = request_naver_api(search_keyword, 1, 1)

        # request_naver_api 함수 호출시 에러 발생하여 None값 반환되면

        if dict_result == None:
            count_test_error += 1  # count 1 증가
            if count_test_error < 4:  # 3회 이하 발생시            
                print('Retry row-index : %s (Test Request) | retry count : %s' % i, count_test_error)
                i -= 1  # 재실행 (i -> 1 감소)
                continue
            else:  # 4회 이상 발생시
                print('Rejected More Than 3 times !!\n')
                # naver_search_rejected에 저장
                naver_search_rejected = naver_search_rejected.append(DataFrame({
                      '지역' : public_salon.iloc[i]['지역']
                    , '업소명' : public_salon.iloc[i]['업소명']
                    , '소재지도로명' : public_salon.iloc[i]['소재지도로명']
                    , '검색어' : search_keyword
                    , '검색건수' : total_count
                    , '원인' : 'test request 3회 실패'
                    }
                    , index = [i]
                ))
                skip_history+=1
                print('Skipped [%s] %s' % (i, search_keyword))
                print('Final Status : Skip Keyword Cause by Retry More Than 3 Times !! (Skip History : %s)' % skip_history)
                continue

        # 총 검색결과 수 뽑기
        total_count = dict_result['total'] 
        print('검색 결과 수 :', total_count)

        # 검색 결과가 없으면
        if total_count == 0:
            naver_search_rejected = naver_search_rejected.append(DataFrame({
                  '지역' : public_salon.iloc[i]['지역']
                , '업소명' : public_salon.iloc[i]['업소명']
                , '소재지도로명' : public_salon.iloc[i]['소재지도로명']
                , '검색어' : search_keyword
                , '검색건수' : total_count
                , '원인' : '검색결과 없음'
                }
                , index = [i]
            ))
            
            print('Final Status : No Naver Search Result !! (Skip History : %s)' % skip_history)
            continue

        # 검색 결과가 있으면
        # 검색결과 300건 초과인 경우 이상치로 판단 (트래픽 에러발생 방지)
        elif total_count > 300:
            naver_search_rejected = naver_search_rejected.append(DataFrame({
                  '지역' : public_salon.iloc[i]['지역']
                , '업소명' : public_salon.iloc[i]['업소명']
                , '소재지도로명' : public_salon.iloc[i]['소재지도로명']
                , '검색어' : search_keyword
                , '검색건수' : total_count
                , '원인' : '검색결과 300건 초과'
                }
                , index = [i]
            ))
            print('Final Status : More Than 300 Search Results !! (Skip History : %s)' % skip_history)
            continue

        # 300건 이하면 정상 진행
        else:

            # 최대 요청 개수인 30으로 나누어서 올림 (반올림하면 안됨)
            # 요청할 회수
            request_count = math.ceil(total_count / 30)
            print('요청할 회수 :', request_count)

            # 요청 회수만큼 URL 만들어서 리스트에 저장
            print('<main connection>')
            dict_list_naver=[]
            count_main_error = 0
            for j in range(request_count):
                # 요청회수가 5번 넘어가면 1.5초 슬립
                if j == 5:
                    time.sleep(1.5)

                # request_naver_api 함수 호출
                dict_result = request_naver_api(search_keyword, 1 + 30 * j, 30)

                # request_naver_api 함수 호출시 에러 발생하여 None값 반환되면
                if dict_result == None:
                    count_main_error += 1  # count 1 증가
                    if count_main_error < 4:  # 3회 이하 발생시
                        j -= 1  # 재실행 (j -> 1 감소)
                        print('Retry row-index : %s (Main Request) | retry count : %s' % i, count_main_error)
                        continue
                    else:  # 4회 이상 발생시
                        print('Rejected More Than 3 times !!\n')
                        # naver_search_rejected에 저장
                        naver_search_rejected = naver_search_rejected.append(DataFrame({
                              '지역' : public_salon.iloc[i]['지역']
                            , '업소명' : public_salon.iloc[i]['업소명']
                            , '소재지도로명' : public_salon.iloc[i]['소재지도로명']
                            , '검색어' : search_keyword
                            , '검색건수' : total_count
                            , '원인' : 'main request 3회 실패'
                            }
                            , index = [i]
                        ))
                        skip_history+=1
                        print('Skipped [%s] %s' % (i, search_keyword))
                        break

                # 정상 반환시 dict_list_naver에 append
                dict_list_naver.append(dict_result)

            # 4회 이상 에러난 경우 다음 검색으로 이동
            if count_main_error > 3:
                print('Final Status : Skip Keyword Cause by Retry More Than 3 Times !! (Skip History : %s)' % skip_history)
                continue

            # 검색결과인 'items' key에 해당하는 value 만 뽑기
            item_list_naver = list(map(lambda x: x['items'], dict_list_naver))

            # 리스트에 저장된 각각의 데이터를 DataFrame으로 말기
            df_list_naver = list(map(lambda x: DataFrame(x), item_list_naver))

            # 리스트의 모든 DF를 하나로 합치기
            df_naver = pd.concat(df_list_naver, ignore_index=True, sort=True)

            # 필요한 컬럼으로 재색인
            df_naver = df_naver.reindex(columns = ['title', 'telephone', 'address', 'roadAddress', 'category'])

            # 컬럼명 변경
            df_naver.rename(columns={
                                      'title' :'업소명'
                                    , 'telephone' : '소재지전화번호'
                                    , 'address' : '소재지지번'
                                    , 'roadAddress' : '소재지도로명'
                                    , 'category' : '카테고리'
                                    }, inplace=True)

            # '업소명'에 들어간 '<br>', '</br>' 등 html 태그 지우기
            df_naver['업소명'] = np.array(list(map(lambda x: re.sub('</\w+>|<\w+>','',x), df_naver['업소명'].values)))
            
            df_naver['검색인덱스'] = i
            
            df_naver['검색어'] = search_keyword

            df_naver['검색건수'] = total_count

            df_naver = df_naver[['업소명'
                                 , '소재지전화번호'
                                 , '소재지도로명'
                                 , '소재지지번'
                                 , '카테고리'
                                 , '검색인덱스'
                                 , '검색어'
                                 , '검색건수']]

            # 네이버 검색결과 DataFrame에 추가(저장)
            naver_search_all = naver_search_all.append(df_naver, ignore_index=True)

            # 오늘 날짜 출력
            today = datetime.datetime.now().strftime('%Y-%m-%d')

            # 현재까지의 네이버 검색 결과를 csv 파일로 저장
            naver_search_all.to_csv('./naver_api_seoul_salon_data_%s.csv' % today, sep=',', encoding='utf-8')

            # 검색결과가 300건이 넘어 크롤링 보류한 데이터도 csv 파일로 저장
            naver_search_rejected.to_csv('./naver_api_seoul_salon_rejected_data_%s.csv' % today, sep=',', encoding='utf-8')

            print('Final Status : Naver Search Result SAVE SUCCESS !! (Skip History : %s)' % skip_history)

# 함수 실행

- 초기 세팅
    - (1) start_number에 본인 시작번호 대입
    - (2) end_number에 본인 끝번호 대입
    - (3) restart_number는 건드릴 필요 없음. 초기값 0 으로 둘 것


- 에러 발생시
    - (1) 가장 마지막 메세지를 복사 -> 별도 셀에 주석으로 저장
    - (2) restart_number 에 에러가 난 try row-index를  대입 후 재실행 할 것


- 총 검색할 개수 15,914 (0 ~ 15,913)
- 16,000 / 5 = 3,200 -> 한 사람당 3,200 개씩 작업
- 0 ~ 3200       : 서동원
- 3201 ~ 6400    : 성민호
- 6401 ~ 9600    : 이한울
- 9601 ~ 12800   : 한주리
- 12801 ~ 15913  : 현택호


In [ ]:
# 본인 시작번호, 끝번호, 재시작번호 기입 후 실행
start_number = 0  # 시작번호
end_number = 3200  # 끝번호

restart_number = 0  # 재시작번호 (초기값 = 0 -> 중지된 가장 마지막 'try row-index' 값 입력)

# 함수 실행
run_naver_api_search(start_number, end_number, restart_number)


















  0%|                                                                                         | 0/3201 [00:00<?, ?it/s]

Try row-index : 0
검색어 : 서울시 강남구 백민재헤어샵 미용실
<test connection>
[2020-04-11 18:30:46.502258] URL Request SUCCESS
검색 결과 수 : 0
Final Status : No Naver Search Result !! (Skip History : 0)



















  0%|                                                                               | 1/3201 [00:01<1:27:05,  1.63s/it]

Try row-index : 1
검색어 : 서울시 강남구 백민재헤어샵 미용실
<test connection>
[2020-04-11 18:30:48.998494] URL Request SUCCESS
검색 결과 수 : 0
Final Status : No Naver Search Result !! (Skip History : 0)



















  0%|                                                                               | 2/3201 [00:04<1:40:50,  1.89s/it]

...


















  1%|▋                                                                             | 30/3201 [01:07<2:21:23,  2.68s/it]

Try row-index : 30
검색어 : 서울시 강남구 누벨바그헤어살롱 미용실
<test connection>
[2020-04-11 18:31:53.691327] URL Request SUCCESS
검색 결과 수 : 1
요청할 회수 : 1
<main connection>
[2020-04-11 18:31:53.807511] URL Request SUCCESS
Final Status : Naver Search Result SAVE SUCCESS !! (Skip History : 0)



















  1%|▊                                                                             | 31/3201 [01:08<1:58:48,  2.25s/it]

Try row-index : 31
검색어 : 서울시 강남구 사랑방미용실 미용실
<test connection>
[2020-04-11 18:31:55.085725] URL Request SUCCESS
검색 결과 수 : 1
요청할 회수 : 1
<main connection>
[2020-04-11 18:31:55.192391] URL Request SUCCESS
Final Status : Naver Search Result SAVE SUCCESS !! (Skip History : 0)



















  1%|▊                                                                             | 32/3201 [01:10<1:45:02,  1.99s/it]

In [40]:
# 에러가 발생한 마지막 메세지 복사 (error_history)
# start 는 별도 기입할 것

# start : 0
# 3200it [2:02:18,  2.58s/it]
# try row-index : 3200
# 검색어 : 서울 진미용실 미용실
# <test connection>
# [2020-04-09 00:02:10.065388] URL Request SUCCESS
# 검색 결과 수 : 97
# 요청할 회수 : 4
# <main connection>
# [2020-04-09 00:02:10.201034] URL Request SUCCESS
# [2020-04-09 00:02:10.288759] URL Request SUCCESS
# [2020-04-09 00:02:10.363592] URL Request SUCCESS
# [2020-04-09 00:02:10.434883] URL Request SUCCESS
# Naver Search Result SAVE SUCCESS !!

# 3201it [2:02:20,  2.47s/it]

# 에러 없이 0-3200 번대 실행 완료!


In [41]:
# 네이버 검색결과 확인
df_naver

NameError: name 'df_naver' is not defined

In [ ]:
# 정상적으로 검색된 데이터 수
len(naver_search_all)

In [ ]:
# 검색결과 300건 넘어 검색 보류한 데이터 수
len(naver_search_rejected)

In [ ]:
# 300건 이상 데이터 확인
naver_search_rejected

In [ ]:
# 정상 검색결과를 모두 저장한 DF 확인
naver_search_all

In [ ]:
naver_search_all['카테고리'].unique()